# Milli-kLockStat Analysis
## To start, run the following cell, replacing the filename and sample period (if necessary).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv

# The sample period in ns. Will typically be 50000000 (50ms).
sample_period_ns = 50000000
sample_period_ms = sample_period_ns // 1000000

# The filename to read data from.
filename = "LOG_NAME.log"
# Add any commands that you want filtered here.
filtered_commands = ["collect", "collectl", "tcplife_bpfcc"]

# Stats Collection
## Run the following cell to read the file and collect statistics for all the remaining cells.

In [ ]:
class Sample:
    def __init__(self, comm, ptids, locktype, lockaddr, ts, totalwait, count, traceid):
        self.comm = comm.strip()
        self.ptids = ptids.strip()
        self.locktype = locktype.strip()
        self.lockaddr = lockaddr.strip()
        self.ts = int(ts)
        self.totalwait = int(totalwait)
        self.count = int(count)
        self.avgwait = self.totalwait / self.count
        self.traceid = int(traceid)

earliest_timestamp = [0x0FFF_FFFF_FFFF_FFFF]
latest_timestamp = [0]
samples = [] # Every single sample.
samples_by_timestamp = {} # Key: a timestamp. Value: a list of indices into the sample array.
samples_by_addr = {} # Key: a lock address. Value: a list of indices into the sample array.
total_access_counts_per_addr = {} # Key: a lock address. Value: the total number of accesses for that lock across all samples.
accumulated_access_time_per_addr = {} # Key: a lock address. Value: the sum of all access times for that lock across all samples.
worst_access_times_per_addr = {} # Key: a lock address. Value: the worst average access time for a period across all samples.
total_access_counts_per_addr_then_timestamp = {} # First key: the lock address. Second key: the timestamp. Value: the sum of all access counts across all threads for one sample.
accumulated_access_time_per_addr_then_timestamp = {} # First key: the lock address. Second key: the timestamp. Value: the sum of all access times across all threads for one sample.
contenders_by_addr = {} # Key: the lock address. Value: the set of process/thread IDs that accessed that lock over the entire log.
contenders_by_addr_then_timestamp = {} # First key: the lock address. Second key: the timestamp. Value: the set of process/thread IDs that accessed that lock over the entire log.
worst_access_times_by_timestamp = {} # Key: a timestamp of a period. Value: the worst AAT experienced by any lock for that period.
first_samples_by_lock_then_ptid = {} # First key: a lock address. Second key: a process+thread ID. Value: a sample index.

traces = {}

def safe_append(d, k, v):
    if not k in d.keys():
        d[k] = [v]
    else:
        d[k].append(v)

def safe_setadd(d, k, v):
    if not k in d.keys():
        d[k] = set([v])
    else:
        d[k].add(v)

def safe_add(d, k, v):
    if not k in d.keys():
        d[k] = v
    else:
        d[k] += v

def double_safe_add(d, k1, k2, v):
    if not k1 in d.keys():
        d[k1] = {}
    if not k2 in d[k1].keys():
        d[k1][k2] = v
    else:
        d[k1][k2] += v

def double_safe_setadd(d, k1, k2, v):
    if not k1 in d.keys():
        d[k1] = {}
    if not k2 in d[k1].keys():
        d[k1][k2] = set([v])
    else:
        d[k1][k2].add(v)

def safe_max(d, k, v):
    if not k in d.keys():
        d[k] = v
    else:
        d[k] = max(d[k], v)

def safe_get(d, k1, default):
    if k1 not in d.keys():
        return default
    else:
        return d[k1]

def safe_double_get(d, k1, k2, default):
    if k1 not in d.keys():
        return default
    elif k2 not in d[k1].keys():
        return default
    else:
        return d[k1][k2]

def safe_double_emplace(d, k1, k2, v):
    if not k1 in d.keys():
        d[k1] = {}
    if not k2 in d[k1].keys():
        d[k1][k2] = v

def add_sample(line):
    sample = Sample(line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7])
    if not (sample.comm in filtered_commands):
        latest_timestamp[0] = max(latest_timestamp[0], sample.ts)
        earliest_timestamp[0] = min(earliest_timestamp[0], sample.ts)
        samples.append(sample)
        i = len(samples) - 1
        safe_append(samples_by_timestamp, sample.ts, i)
        safe_append(samples_by_addr, sample.lockaddr, i)
        safe_add(total_access_counts_per_addr, sample.lockaddr, sample.count)
        safe_add(accumulated_access_time_per_addr, sample.lockaddr, sample.totalwait)
        safe_max(worst_access_times_per_addr, sample.lockaddr, sample.avgwait)
        double_safe_add(total_access_counts_per_addr_then_timestamp, sample.lockaddr, sample.ts, sample.count)
        double_safe_add(accumulated_access_time_per_addr_then_timestamp, sample.lockaddr, sample.ts, sample.totalwait)
        safe_setadd(contenders_by_addr, sample.lockaddr, sample.ptids)
        double_safe_setadd(contenders_by_addr_then_timestamp, sample.lockaddr, sample.ts, sample.ptids)
        safe_max(worst_access_times_by_timestamp, sample.ts, sample.avgwait)
        safe_double_emplace(first_samples_by_lock_then_ptid, sample.lockaddr, sample.ptids, i)

file = open(filename)
lines = csv.reader(file)
for line in lines:
    if len(line) > 0:
        if line[0] == "s":
            add_sample(line[1:])
        elif line[0] == "t":
            traces[int(line[1])] = line[2:]
file.close()

# Modify these if you wish to use custom timestamps for graphing
latest_timestamp = latest_timestamp[0]
earliest_timestamp = earliest_timestamp[0]

# Print Most Contended Locks
## Run this cell to list the locks that have been accessed by the most threads. 

In [ ]:
most_contended_locks = []
for addr in samples_by_addr.keys():
    most_contended_locks.append((len(contenders_by_addr[addr]), addr))
most_contented_locks = sorted(most_contended_locks, reverse=True)
print("20 most contended locks (sorted by highest number of unique threads that accessed each lock)")
for aat, addr in most_contented_locks[:20]:
    print(f"{addr}\t{aat} threads")

# Graph Contention For Individual Addresses
## Run this cell to graph the number of threads simultaneously accessing a lock over time.

In [ ]:
lock = "0xLOCKADDRESSHERE"

# Modify these values to shrink the time span shown in the graph
# graph_earliest_timestamp = earliest_timestamp
graph_earliest_timestamp = latest_timestamp - 720 * 1000000000
graph_latest_timestamp = latest_timestamp

graph_earliest_timestamp = (graph_earliest_timestamp // sample_period_ns) * sample_period_ns
graph_latest_timestamp = (graph_latest_timestamp // sample_period_ns) * sample_period_ns
valid_timestamps = np.arange(graph_earliest_timestamp, graph_latest_timestamp + sample_period_ns, sample_period_ns)
aats = np.array([len(safe_get(contenders_by_addr_then_timestamp[lock], i, set([]))) for i in valid_timestamps]) / 1000000 # convert to ms
easy_to_read_timestamps = (valid_timestamps - graph_earliest_timestamp) // 1000000000 # convert to s
plt.rcParams["figure.figsize"] = (30,5)
plt.xlabel("Time (s)")
plt.ylabel("Worst Wait Time (ms)")
plt.plot(easy_to_read_timestamps, aats)

# Print Threads Accessing a Lock
## Run this cell to list the threads, processes, and commands that are using a specific lock, along with stack traces for each thread.

In [ ]:
lock = "0xLOCKADDRESSHERE"
first_samples_by_ptid = first_samples_by_lock_then_ptid[lock]
for sampleid in first_samples_by_ptid.values():
    sample = samples[sampleid]
    print(f"Comm: {sample.comm}, PTIDS: {sample.ptids}, Trace: {traces[sample.traceid]}")

# Print Worst Locks by AAT
## Run this cell to list the locks that experienced the worse average access time over the entire thread.
Note: this is calculated by summing the total wait time across all accesses for the entire log, divided by the total number of accesses for the entire log.

In [ ]:
# Set this number to be larger than 1 if you want to filter out locks that were only used by 1 thread.
min_required_threads_using = 1

duration = (latest_timestamp - earliest_timestamp) / 1e9
print(f"Timestamp span: {earliest_timestamp}-{latest_timestamp} ({duration} seconds)")

worst_total_aats = []
for addr in samples_by_addr.keys():
    if len(contenders_by_addr[addr]) >= min_required_threads_using:
        worst_total_aats.append((accumulated_access_time_per_addr[addr] / total_access_counts_per_addr[addr], addr))
worst_total_aats = sorted(worst_total_aats, reverse=True)
print("20 worst locks by AAT, averaged over the entire sampling time")
for aat, addr in worst_total_aats[:20]:
    print(f"{addr}\t{aat} ns")

# Print Worst Locks by Worst Period AAT
## Run this cell to list the locks that experienced the worst average latency for some 50ms period during the log.
This is useful for detecting the worst spikes.

In [ ]:
# Set this number to be larger than 1 if you want to filter out locks that were only used by 1 thread.
min_required_threads_using = 1

locks_to_print = 10

worst_single_sample_aats = []
for addr in samples_by_addr.keys():
    if len(contenders_by_addr[addr]) >= min_required_threads_using:
        worst_single_sample_aats.append((worst_access_times_per_addr[addr], addr))
worst_single_sample_aats = sorted(worst_single_sample_aats, reverse=True)
print(f"{locks_to_print} worst locks, based on the worst AAT observed by a thread using the lock over some {sample_period_ms}ms period")
for aat, addr in worst_single_sample_aats[:locks_to_print]:
    print(f"{addr}\t{aat} ns")

# Graph Worst Waits for Any Lock
## Run this cell to graph the worst average wait time for ANY lock for all sample periods.
This is useful to locating particularly bad contention spikes.

In [ ]:
# Modify these values to shrink the time span shown in the graph
# graph_earliest_timestamp = earliest_timestamp
graph_earliest_timestamp = latest_timestamp - 720 * 1000000000
graph_latest_timestamp = latest_timestamp

graph_earliest_timestamp = (graph_earliest_timestamp // sample_period_ns) * sample_period_ns
graph_latest_timestamp = (graph_latest_timestamp // sample_period_ns) * sample_period_ns
valid_timestamps = np.arange(graph_earliest_timestamp, graph_latest_timestamp + sample_period_ns, sample_period_ns)
aats = np.array([safe_get(worst_access_times_by_timestamp, i, 0) for i in valid_timestamps]) / 1000000 # convert to ms
easy_to_read_timestamps = (valid_timestamps - graph_earliest_timestamp) // 1000000000 # convert to s
plt.rcParams["figure.figsize"] = (30,5)
plt.xlabel("Time (s)")
plt.ylabel("Worst Wait Time (ms)")
plt.plot(easy_to_read_timestamps, aats)

# Individual Lock Graphing
## Run this cell to graph the average lock wait time over the course of the entire log, for a specific lock.

In [ ]:
# Modify this to check a different lock
lock_to_graph = "0xLOCKADDRESSHERE"

# Modify these values to shrink the time span shown in the graph
# graph_earliest_timestamp = earliest_timestamp
graph_earliest_timestamp = latest_timestamp - 720 * 1000000000
graph_latest_timestamp = latest_timestamp

graph_earliest_timestamp = (graph_earliest_timestamp // sample_period_ns) * sample_period_ns
graph_latest_timestamp = (graph_latest_timestamp // sample_period_ns) * sample_period_ns
valid_timestamps = np.arange(graph_earliest_timestamp, graph_latest_timestamp + sample_period_ns, sample_period_ns)
acc_acc_times = accumulated_access_time_per_addr_then_timestamp[lock_to_graph]
tot_acc_counts = total_access_counts_per_addr_then_timestamp[lock_to_graph]
aats = [safe_get(acc_acc_times, i, 0) / safe_get(tot_acc_counts, i, 1) for i in valid_timestamps]
easy_to_read_timestamps = (valid_timestamps - graph_earliest_timestamp) // 1000000000
plt.rcParams["figure.figsize"] = (30,5)
plt.xlabel("Time (s)")
plt.ylabel("Worst Wait Time (ms)")
plt.plot(easy_to_read_timestamps, aats)